<a href="https://colab.research.google.com/github/showtimeon0524/ZoRrO/blob/master/Stable_Diffusion_WebUI_Fast_Deployment_V2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion Web-UI Colab Quick Deployment V2.0

# AI 作图一键部署

---
## 感谢大佬的开源！

> 原完整项目：https://github.com/wibus-wee/stable_diffusion_chilloutmix_ipynb
>
> 个人在此基础上做了精简与重启改良 @[Lightship](https://space.bilibili.com/440840365)
>
>
> 祝大家玩的开心！

---
## 运行方法：

> 首次使用配置环境并启动：
> 1. 如果不使用 Google Drive，请跳过0块，直接运行1、2、3、4块
> 2. 如果使用 Google Drive，请全部运行0、1、2、3、4块，Checkpoint 与 Lora 的配置方法相同。

> 再次使用（指关闭colab后重新连接容器实例）：
> 1. 如果部署时使用过 Google Drive，那么直接运行0、1、4块，启动时间短很多。
> 2. 如果没有使用过 Google Drive，请按首次使用进行环境配置。
---

## 0.装载 Google Drive
运行并授权后，会将 Stable Diffusion 部署于 Google Drive 中
> 这样做的好处是：
> 1. 下次使用可以无需重新安装依赖和模型，直接运行0、1、4块即可。
> 2. 如果web界面卡图导致这次的图没出来，除了恢复种子重新尝试之外，也可以在 drive 中查看生成的图（路径：'drive/MyDrive/stable-diffusion-webui/log/images'，或者直接进drive看最近操作文件）。

> 这样的缺点是：
> 1. drive中的免费额度只有15G，如果空间不够将跑不起来，因此这种方式限制了模型的大小和数量。如果发现有报错，就去看看自己的drive，该清理的删一删，记得清空回收站，然后把第一块的模型注释掉一部分，重新从头跑一次。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
useDrive = True
%store useDrive

## 1.配置Checkpoint（可自定义） + 检查GPU

In [ ]:
# 列在downloadLink中的Checkpoint会自动下载（更改配置后需要运行第二块），可以自由增减，并在部署完成后，于WebUI中选择其中一个
CheckpointLink = {
    'chilloutmix.safetensors': 'https://huggingface.co/swl-models/chilloutmix/resolve/main/Chilloutmix-ema-fp32.safetensors',
    # 'sunshinemix.safetensors': 'https://civitai.com/api/download/models/11752',
    # 'grapefruitHentaiModel.safetensors': 'https://civitai.com/api/download/models/16619',
    # 'meinaMix.safetensors': 'https://civitai.com/api/download/models/13434',
}

%store CheckpointLink

# 检查GPU
import os, subprocess
paperspace_m4000 = False
try:
    subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE)
    gpuinfo = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    if 'M4000' in gpuinfo:
        print("WARNING: 你正在使用的是 Quadro M4000 GPU，它将无法使用 xformers。")
        paperspace_m4000 = True
    else:
        print("当前使用："+gpuinfo.strip())
        print("你正在使用的是合适的 GPU。")
except:
    print("似乎没有 GPU 可用。请检查你的运行时类型。")
    exit()

rootDir = '/tmp' if paperspace_m4000 else '/content'

useDrive = False
%store -r useDrive
if useDrive:
  rootDir = rootDir+'/drive/MyDrive'
  print('你正在使用drive，请确保空间足够！')

%store rootDir
%store paperspace_m4000 

## 2.安装依赖与模型

In [ ]:
%store -r paperspace_m4000 
%store -r rootDir 
%store -r CheckpointLink

import os, subprocess
from glob import glob
from pathlib import Path
import sys

!apt-get -y install -qq aria2
ariaInstalled = False

try:
    subprocess.run(['aria2c', '--version'], stdout=subprocess.PIPE)
    ariaInstalled = True
except:
    pass

%store ariaInstalled

if paperspace_m4000:
  # Uninstall PyTorch and some other libraries so the WebUI can install the versions it needs
  !pip uninstall -y torch torchvision torchaudio protobuf
  !pip install protobuf
  !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
else:
  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  !pip install -q --pre triton
  

!git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui {rootDir}/stable-diffusion-webui
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O {rootDir}/stable-diffusion-webui/scripts/run_n_times.py
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {rootDir}/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {rootDir}/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface {rootDir}/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone -b v2.0 https://github.com/camenduru/sd-civitai-browser {rootDir}/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks {rootDir}/stable-diffusion-webui/extensions/sd-webui-additional-networks

if paperspace_m4000:
  %cd /stable-diffusion-webui
else:
  %cd {rootDir}/stable-diffusion-webui
  
for checkpointName in CheckpointLink:
  if ariaInstalled:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {CheckpointLink[checkpointName]} -d {rootDir}/stable-diffusion-webui/models/Stable-diffusion -o {checkpointName}
  else:
      !wget -c {CheckpointLink[checkpointName]} -P {rootDir}/stable-diffusion-webui/models/Stable-diffusion -O {rootDir}/stable-diffusion-webui/models/Stable-diffusion/{checkpointName}



## 3.导入Lora模型（可自定义）

In [ ]:
%store -r rootDir
%store -r ariaInstalled
%store -r paperspace_m4000
import os
# launch运行前不会生成Lora目录，提前创建即可
!mkdir {rootDir}/stable-diffusion-webui/models/Lora

# 对于找到的Lora模型，将其名字与下载连接放入如下字典即可
# 这个名称就是出图使用<lora:文件名:权重>载入的文件名
LoraLink = {
    'koreanDollLikeness_v15': 'https://huggingface.co/Anon1216/kdllora-v1.5/resolve/main/koreanDollLikeness_v15.safetensors',
    'japaneseDollLikeness_v10': 'https://huggingface.co/AmornthepKladmee/japaneseDollLikeness_v10/resolve/main/japaneseDollLikeness_v10.safetensors',
    'chilloutmixss_xss10': 'https://civitai.com/api/download/models/12876',
    'aronaBlueArchive': 'https://civitai.com/api/download/models/9409',
}

if paperspace_m4000:
  %cd /stable-diffusion-webui
else:
  %cd {rootDir}/stable-diffusion-webui
  

for loraName in LoraLink:
  path = rootDir+'/stable-diffusion-webui/models/Lora/'+loraName+'.safetensors'
  if os.path.exists(path):
    print('已经导入Lora：'+loraName)
    continue
  if ariaInstalled:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {LoraLink[loraName]} -d {rootDir}/stable-diffusion-webui/models/Lora -o {loraName}.safetensors
  else:
      !wget -c {LoraLink[loraName]} -P {rootDir}/stable-diffusion-webui/models/Lora -O {rootDir}/stable-diffusion-webui/models/Lora/{loraName}.safetensors
  print("成功导入Lora："+loraName)

print('当前可使用的Lora名称：')
for loraName in LoraLink:
  print(loraName)


## 4.启动

In [ ]:
#@title
%store -r paperspace_m4000 
%store -r rootDir 

%cd {rootDir}/stable-diffusion-webui
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\\[t\\]/self.logvar\\[t.item()\\]/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!sed -i '/Checking out commit for/d' {rootDir}/stable-diffusion-webui/launch.py
!python launch.py --share --enable-insecure-extension-access {'' if paperspace_m4000 else '--xformers'}